In [1]:
import numpy
import cirq
import openfermion

import itertools

In [2]:
from openfermion import FermionOperator

In [3]:
def append_operators(operator, n_electrons, n_qubits, only_singles=False):

    """
    Add all operators that are not included. For both spin conserving and 
    not spin conserving
    
    Args:
        operator (FermionOperator):  Operator to which is added.
        n_electrons (int):           Number electrons in system.
        n_qubits (int):              Number of qubits.
        only_singles(Boolean):       Add only singels or singles and doubles
        
    Returns:
        ucc_spin, ucc_all (FerminOperator): Complete spin conserving operator and
        complete operator
        
    Notes:
        This function add the missing operators with a value of 1.0
    """

    ucc_all = FermionOperator()
    ucc_spin =FermionOperator()
    for j in range(0,n_electrons):
        for b in range(n_electrons,n_qubits): 
            temp_op_add = FermionOperator('{}^ {}'.format(b,j), 1.0)
            if list(temp_op_add.terms.keys())[0] not in list(operator.terms.keys()):
                ucc_all += temp_op_add
                ucc_all += openfermion.hermitian_conjugated(temp_op_add)
                if (b%2 == j%2):
                    ucc_spin += temp_op_add
                    ucc_spin += openfermion.hermitian_conjugated(temp_op_add)
            if not(only_singles):
                for i in range(0+j,n_electrons):  
                    for a in range(b,n_qubits):
                        temp_op_add = FermionOperator('{}^ {}^ {} {}'.format(a,b,i,j), 1.0)
                        if list(temp_op_add.terms.keys())[0] not in list(operator.terms.keys()):
                            ucc_all += temp_op_add
                            ucc_all += openfermion.hermitian_conjugated(temp_op_add)
                            if (a%2 == i%2)*(b%2 == j%2) or (a%2 == j%2)*(b%2 == i%2):
                                ucc_spin += temp_op_add
                                ucc_spin += openfermion.hermitian_conjugated(temp_op_add)
    ucc_all +=operator
    ucc_spin+=operator
    return openfermion.normal_ordered(ucc_spin), openfermion.normal_ordered(ucc_all)

In [4]:
fop_mart = append_operators(openfermion.FermionOperator(), 4,8)[1]

In [40]:
def _generate_ucc_amplitudes_spin_conserved(n_electrons, n_orbitals):
    """
    Create list of amplitudes to generate UCC operators.
    
    n_electrons (int): Number of electrons.
    n_orbitals (int): Number of spin-orbitals, equivalent to number of qubits.
    
    Returns:
        single_amplitudes, double_amplitudes (list): List of single
            and double amplitudes as [[i,j], t_ij, [k,l], t_kl]
    """
    single_amplitudes = []
    double_amplitudes = []
    for one_el in range(0, n_electrons):
        for unocc_orb in range(n_electrons, n_orbitals):
            if (unocc_orb%2 == one_el%2):
                single_amplitudes.append([[unocc_orb, one_el], 1.0])
            for two_el in range(one_el, n_electrons):
                for two_unocc_orb in range(unocc_orb, n_orbitals):
                    if (((two_unocc_orb%2 == two_el%2) and
                        (unocc_orb%2 == one_el%2) or
                        (two_unocc_orb%2 == one_el%2) and
                        (unocc_orb%2 == two_el%2)) and
                        (two_unocc_orb!=unocc_orb) and 
                        (two_el!=one_el)):
                            double_amplitudes.append([[two_el, two_unocc_orb, one_el, unocc_orb], 1.0])

    return single_amplitudes, double_amplitudes

In [41]:
sing, doubs  = _generate_ucc_amplitudes_spin_conserved(4, 8)

In [42]:
print(sing, doubs)

[[[4, 0], 1.0], [[6, 0], 1.0], [[5, 1], 1.0], [[7, 1], 1.0], [[4, 2], 1.0], [[6, 2], 1.0], [[5, 3], 1.0], [[7, 3], 1.0]] [[[1, 5, 0, 4], 1.0], [[1, 7, 0, 4], 1.0], [[2, 6, 0, 4], 1.0], [[3, 5, 0, 4], 1.0], [[3, 7, 0, 4], 1.0], [[1, 6, 0, 5], 1.0], [[3, 6, 0, 5], 1.0], [[1, 7, 0, 6], 1.0], [[3, 7, 0, 6], 1.0], [[2, 5, 1, 4], 1.0], [[2, 7, 1, 4], 1.0], [[2, 6, 1, 5], 1.0], [[3, 7, 1, 5], 1.0], [[2, 7, 1, 6], 1.0], [[3, 5, 2, 4], 1.0], [[3, 7, 2, 4], 1.0], [[3, 6, 2, 5], 1.0], [[3, 7, 2, 6], 1.0]]


In [43]:
numpy.shape(sing)

(8, 2)

In [44]:
doubs

[[[1, 5, 0, 4], 1.0],
 [[1, 7, 0, 4], 1.0],
 [[2, 6, 0, 4], 1.0],
 [[3, 5, 0, 4], 1.0],
 [[3, 7, 0, 4], 1.0],
 [[1, 6, 0, 5], 1.0],
 [[3, 6, 0, 5], 1.0],
 [[1, 7, 0, 6], 1.0],
 [[3, 7, 0, 6], 1.0],
 [[2, 5, 1, 4], 1.0],
 [[2, 7, 1, 4], 1.0],
 [[2, 6, 1, 5], 1.0],
 [[3, 7, 1, 5], 1.0],
 [[2, 7, 1, 6], 1.0],
 [[3, 5, 2, 4], 1.0],
 [[3, 7, 2, 4], 1.0],
 [[3, 6, 2, 5], 1.0],
 [[3, 7, 2, 6], 1.0]]

In [45]:
len(openfermion.normal_ordered(openfermion.uccsd_generator([], doubs)).terms)

36

In [46]:
len(openfermion.normal_ordered(openfermion.uccsd_generator(sing, doubs)).terms)

52

In [50]:
uccs_spin_fop = []
uccd_spin_fop = []

In [51]:
uccs_spin_fop += [openfermion.normal_ordered(openfermion.uccsd_generator([op], [])) for op in sing]
#uccd_spin_fop = openfermion.normal_ordered(openfermion.uccsd_generator([], doubs))

In [52]:
uccs_spin_fop

[-1.0 [0^ 4] +
 1.0 [4^ 0],
 -1.0 [0^ 6] +
 1.0 [6^ 0],
 -1.0 [1^ 5] +
 1.0 [5^ 1],
 -1.0 [1^ 7] +
 1.0 [7^ 1],
 -1.0 [2^ 4] +
 1.0 [4^ 2],
 -1.0 [2^ 6] +
 1.0 [6^ 2],
 -1.0 [3^ 5] +
 1.0 [5^ 3],
 -1.0 [3^ 7] +
 1.0 [7^ 3]]

In [170]:
openfermion.is_hermitian(ucc_spin_fop)

False

In [171]:
ucc_spin_fop

1.0 [0^ 4] +
1.0 [0^ 6] +
1.0 [1^ 5] +
1.0 [1^ 7] +
1.0 [2^ 4] +
1.0 [2^ 6] +
1.0 [3^ 5] +
1.0 [3^ 7] +
-1.0 [4^ 0] +
1.0 [4^ 0^ 5 1] +
1.0 [4^ 0^ 5 3] +
1.0 [4^ 0^ 6 2] +
1.0 [4^ 0^ 7 1] +
1.0 [4^ 0^ 7 3] +
1.0 [4^ 1^ 5 2] +
1.0 [4^ 1^ 7 2] +
-1.0 [4^ 2] +
1.0 [4^ 2^ 5 3] +
1.0 [4^ 2^ 7 3] +
1.0 [5^ 0^ 6 1] +
1.0 [5^ 0^ 6 3] +
-1.0 [5^ 1] +
-1.0 [5^ 1^ 4 0] +
1.0 [5^ 1^ 6 2] +
1.0 [5^ 1^ 7 3] +
-1.0 [5^ 2^ 4 1] +
1.0 [5^ 2^ 6 3] +
-1.0 [5^ 3] +
-1.0 [5^ 3^ 4 0] +
-1.0 [5^ 3^ 4 2] +
-1.0 [6^ 0] +
1.0 [6^ 0^ 7 1] +
1.0 [6^ 0^ 7 3] +
-1.0 [6^ 1^ 5 0] +
1.0 [6^ 1^ 7 2] +
-1.0 [6^ 2] +
-1.0 [6^ 2^ 4 0] +
-1.0 [6^ 2^ 5 1] +
1.0 [6^ 2^ 7 3] +
-1.0 [6^ 3^ 5 0] +
-1.0 [6^ 3^ 5 2] +
-1.0 [7^ 1] +
-1.0 [7^ 1^ 4 0] +
-1.0 [7^ 1^ 6 0] +
-1.0 [7^ 2^ 4 1] +
-1.0 [7^ 2^ 6 1] +
-1.0 [7^ 3] +
-1.0 [7^ 3^ 4 0] +
-1.0 [7^ 3^ 4 2] +
-1.0 [7^ 3^ 5 1] +
-1.0 [7^ 3^ 6 0] +
-1.0 [7^ 3^ 6 2]

In [161]:
fop_mart

1.0 [0^ 4] +
1.0 [0^ 5] +
1.0 [0^ 6] +
1.0 [0^ 7] +
1.0 [1^ 0^ 5 4] +
1.0 [1^ 0^ 6 4] +
1.0 [1^ 0^ 6 5] +
1.0 [1^ 0^ 7 4] +
1.0 [1^ 0^ 7 5] +
1.0 [1^ 0^ 7 6] +
1.0 [1^ 4] +
1.0 [1^ 5] +
1.0 [1^ 6] +
1.0 [1^ 7] +
1.0 [2^ 0^ 5 4] +
1.0 [2^ 0^ 6 4] +
1.0 [2^ 0^ 6 5] +
1.0 [2^ 0^ 7 4] +
1.0 [2^ 0^ 7 5] +
1.0 [2^ 0^ 7 6] +
1.0 [2^ 1^ 5 4] +
1.0 [2^ 1^ 6 4] +
1.0 [2^ 1^ 6 5] +
1.0 [2^ 1^ 7 4] +
1.0 [2^ 1^ 7 5] +
1.0 [2^ 1^ 7 6] +
1.0 [2^ 4] +
1.0 [2^ 5] +
1.0 [2^ 6] +
1.0 [2^ 7] +
1.0 [3^ 0^ 5 4] +
1.0 [3^ 0^ 6 4] +
1.0 [3^ 0^ 6 5] +
1.0 [3^ 0^ 7 4] +
1.0 [3^ 0^ 7 5] +
1.0 [3^ 0^ 7 6] +
1.0 [3^ 1^ 5 4] +
1.0 [3^ 1^ 6 4] +
1.0 [3^ 1^ 6 5] +
1.0 [3^ 1^ 7 4] +
1.0 [3^ 1^ 7 5] +
1.0 [3^ 1^ 7 6] +
1.0 [3^ 2^ 5 4] +
1.0 [3^ 2^ 6 4] +
1.0 [3^ 2^ 6 5] +
1.0 [3^ 2^ 7 4] +
1.0 [3^ 2^ 7 5] +
1.0 [3^ 2^ 7 6] +
1.0 [3^ 4] +
1.0 [3^ 5] +
1.0 [3^ 6] +
1.0 [3^ 7] +
1.0 [4^ 0] +
1.0 [4^ 1] +
1.0 [4^ 2] +
1.0 [4^ 3] +
1.0 [5^ 0] +
1.0 [5^ 1] +
1.0 [5^ 2] +
1.0 [5^ 3] +
1.0 [5^ 4^ 1 0] +
1.0 [5^ 4^ 2 0] +
1.0 

In [162]:
ucc_spin_qop = openfermion.jordan_wigner(ucc_spin_fop)

In [129]:
ucc_spin_qop

-0.125j [X0 X1 X4 Y5] +
-0.125j [X0 X1 X4 Z5 Z6 Y7] +
0.125j [X0 X1 Y4 X5] +
0.125j [X0 X1 Y4 Z5 Z6 X7] +
-0.125j [X0 X1 X5 Y6] +
0.125j [X0 X1 Y5 X6] +
-0.125j [X0 X1 X6 Y7] +
0.125j [X0 X1 Y6 X7] +
-0.125j [X0 Y1 X4 X5] +
-0.125j [X0 Y1 X4 Z5 Z6 X7] +
-0.125j [X0 Y1 Y4 Y5] +
-0.125j [X0 Y1 Y4 Z5 Z6 Y7] +
-0.125j [X0 Y1 X5 X6] +
-0.125j [X0 Y1 Y5 Y6] +
-0.125j [X0 Y1 X6 X7] +
-0.125j [X0 Y1 Y6 Y7] +
-0.125j [X0 Z1 X2 X4 Z5 Y6] +
0.125j [X0 Z1 X2 Y4 Z5 X6] +
-0.5j [X0 Z1 Y2] +
-0.125j [X0 Z1 Y2 X4 Z5 X6] +
-0.125j [X0 Z1 Y2 Y4 Z5 Y6] +
0.25j [X0 Z1 Y2 Z4] +
0.25j [X0 Z1 Y2 Z6] +
-0.125j [X0 Z1 Z2 X3 X4 Y5] +
-0.125j [X0 Z1 Z2 X3 X4 Z5 Z6 Y7] +
0.125j [X0 Z1 Z2 X3 Y4 X5] +
0.125j [X0 Z1 Z2 X3 Y4 Z5 Z6 X7] +
-0.125j [X0 Z1 Z2 X3 X5 Y6] +
0.125j [X0 Z1 Z2 X3 Y5 X6] +
-0.125j [X0 Z1 Z2 X3 X6 Y7] +
0.125j [X0 Z1 Z2 X3 Y6 X7] +
-0.125j [X0 Z1 Z2 Y3 X4 X5] +
-0.125j [X0 Z1 Z2 Y3 X4 Z5 Z6 X7] +
-0.125j [X0 Z1 Z2 Y3 Y4 Y5] +
-0.125j [X0 Z1 Z2 Y3 Y4 Z5 Z6 Y7] +
-0.125j [X0 Z1 Z2 Y3 X5 X6] +
-0.1